In [ ]:
!python -m spacy download en_core_web_sm
!python -m spacy download pl_core_news_sm

In [2]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

import gensim
from gensim import corpora
import string
import spacy
# import en_core_web_sm #??

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from tqdm import tqdm

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np

from sklearn.decomposition import PCA
import re 


# Constants

In [3]:
data = pd.read_csv('articles_reduced.csv')
nlp = spacy.load("pl_core_news_sm")

PUNCTUATION = '''!()-[]{};:'"\,<>./?@#$%^&*_~“”’–„'''
STOP_WORDS = ['moment', 'chodzić', 'robić', 'wrażenie', 'faktycznie', 'myśleć', 'chcieć', 'czas', 'zrobić', 'wiedzieć', 'trochę', 'swój', 'super', 'naprawdę','prosty', 'drugi', 'przykład', 'troszeczkę', 'raczej', 'siebie', 'wydawać','rzecz', 'chyba', 'różny', 'a',	'aby',	'ach',	'acz',	'aczkolwiek',	'aj',	'albo',	'ale',	'ależ',	'ani',	'aż',	'bardziej',	'bardzo',	'bez',	'bo',	'bowiem',	'by',	'byli',	'bym',	'bynajmniej',	'być',	'był',	'była',	'było',	'były',	'będzie',	'będą',	'cali',	'cała',	'cały',	'chce',	'choć',	'ci',	'ciebie',	'cię',	'co',	'cokolwiek',	'coraz',	'coś',	'czasami',	'czasem',	'czemu',	'czy',	'czyli',	'często',	'daleko',	'dla',	'dlaczego',	'dlatego',	'do',	'dobrze',	'dokąd',	'dość',	'dr',	'dużo',	'dwa',	'dwaj',	'dwie',	'dwoje',	'dzisiaj',	'dziś',	'gdy',	'gdyby',	'gdyż',	'gdzie',	'gdziekolwiek',	'gdzieś',	'go',	'godz',	'hab',	'i',	'ich',	'ii',	'iii',	'ile',	'im',	'inna',	'inne',	'inny',	'innych',	'inż',	'iv',	'ix',	'iż',	'ja',	'jak',	'jakaś',	'jakby',	'jaki',	'jakichś',	'jakie',	'jakiś',	'jakiż',	'jakkolwiek',	'jako',	'jakoś',	'je',	'jeden',	'jedna',	'jednak',	'jednakże',	'jedno',	'jednym',	'jedynie',	'jego',	'jej',	'jemu',	'jest',	'jestem',	'jeszcze',	'jeśli',	'jeżeli',	'już',	'ją',	'każdy',	'kiedy',	'kierunku',	'kilka',	'kilku',	'kimś',	'kto',	'ktokolwiek',	'ktoś',	'która',	'które',	'którego',	'której',	'który',	'których',	'którym',	'którzy',	'ku',	'lat',	'lecz',	'lub',	'ma',	'mają',	'mam',	'mamy',	'mało',	'mgr',	'mi',	'miał',	'mimo',	'między',	'mnie',	'mną',	'mogą',	'moi',	'moim',	'moja',	'moje','móc', 'mieć',	'może',	'możliwe',	'można',	'mu',	'musi',	'my',	'mój',	'na',	'nad',	'nam',	'nami',	'nas',	'nasi',	'nasz',	'nasza',	'nasze',	'naszego',	'naszych',	'natomiast',	'natychmiast',	'nawet',	'nic',	'nich',	'nie',	'niech',	'niego',	'niej',	'niemu',	'nigdy',	'nim',	'nimi',	'nią',	'niż',	'no',	'nowe',	'np',	'nr',	'o',	'o.o.',	'obok',	'od',	'ok',	'około',	'on',	'ona',	'one',	'oni',	'ono',	'oraz',	'oto',	'owszem',	'pan',	'pana',	'pani',	'pl',	'po',	'pod',	'podczas',	'pomimo',	'ponad',	'ponieważ',	'powinien',	'powinna',	'powinni',	'powinno',	'poza',	'prawie',	'prof',	'przecież',	'przed',	'przede',	'przedtem',	'przez',	'przy',	'raz',	'razie',	'roku',	'również',	'sam',	'sama',	'się',	'skąd',	'sobie',	'sobą',	'sposób',	'swoje',	'są',	'ta',	'tak',	'taka',	'taki',	'takich',	'takie',	'także',	'tam',	'te',	'tego',	'tej',	'tel',	'temu',	'ten',	'teraz',	'też',	'to',	'tobie',	'tobą',	'toteż',	'totobą',	'trzeba',	'tu',	'tutaj',	'twoi',	'twoim',	'twoja',	'twoje',	'twym',	'twój',	'ty',	'tych',	'tylko',	'tym',	'tys',	'tzw',	'tę',	'u',	'ul',	'vi',	'vii',	'viii',	'vol',	'w',	'wam',	'wami',	'was',	'wasi',	'wasz',	'wasza',	'wasze',	'we',	'według',	'wie',	'wiele',	'wielu',	'więc',	'więcej',	'wszyscy',	'wszystkich',	'wszystkie',	'wszystkim',	'wszystko',	'wtedy',	'www',	'wy',	'właśnie',	'wśród',	'xi',	'xii',	'xiii',	'xiv',	'xv',	'z',	'za',	'zapewne',	'zawsze',	'zaś',	'ze',	'zeznowu',	'znowu',	'znów',	'został',	'zł',	'żaden',	'żadna',	'żadne',	'żadnych',	'że',	'żeby']

try:
  stopwords_file = '/content/polish.stopwords.txt'
  with open(stopwords_file, 'r') as f:
    stop_words = f.read().splitlines()
  STOP_WORDS.extend(stop_words)
except:
  pass

lemma = WordNetLemmatizer()

# Functions

In [4]:
def prepare_sentences(text):
    sentences = [] 
    for sentence in text.replace('\n','').split('.'):
        if len(sentence) > 10:
            sentences.append(sentence.strip())
    return sentences

def clean(doc):
    
    words = " ".join([i for i in doc.lower().split()])
    # words = words.translate(str.maketrans('', '', string.punctuation))
    words = nlp(words)
    
    normalized_list = []
    for word in words:
        lemma = lemmatize_word(word)
        if lemma not in STOP_WORDS and lemma not in PUNCTUATION and ' ' not in lemma and lemma:
            normalized_list.append(lemma)

    normalized = ' '.join(normalized_list)  

    # print("NORMALIZED: " + normalized) #testuj tu cos                      
    return normalized #zwraca znormalizwane zdanie po zdaniu 

def lemmatize_word(word):
    lemma = word.lemma_
    if lemma.strip().lower():
        return lemma
    
def get_lda(text, topics, words):
    doc_complete = prepare_sentences(text)
    
    doc_clean = [clean(doc).split() for doc in doc_complete] 

    # Creating the term dictionary of our courpus, where every unique term is assigned an index. 
    dictionary = corpora.Dictionary(doc_clean)

    # Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

    # Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

    # Creating the object for LDA model using gensim library
    Lda = gensim.models.ldamodel.LdaModel

    # Running and Trainign LDA model on the document term matrix.
    ldamodel = Lda(doc_term_matrix, num_topics=topics, id2word = dictionary, passes=50)
    lda = ldamodel.print_topics(num_topics=topics, num_words=words)

    return lda

def topic_modeling(data, content_col, label_col):

    topic_list = []
    labels = []

    # Iterating through content with topic modeling
    for n,article in enumerate(tqdm(data[content_col])):
        if type(article) == str and 'sport' not in article.lower():
            try:
                topics = get_lda(article, 3, 10)
                topic_list.append(topics)

                labels.append(list(data[label_col])[n])
            except:
                pass
            
    return [topic_list, labels]

def generate_keywords(topic_list):
    keywords = []

    # Extract the topic keywords without the likelihoods
    for t_list in topic_list:
        keywords.append([fragment for tup in t_list for fragment in re.findall(r'"([^"]*)"', tup[1])])
        
    return keywords


def generate_similarity_matrix(document_list): 
    global vectorizer 

    # Create the TF-IDF vectorizer
    vectorizer = TfidfVectorizer()

    # Compute the TF-IDF matrix
    tfidf_matrix = vectorizer.fit_transform(document_list)

    # Calculate the cosine similarity matrix
    cosine_similarities = cosine_similarity(tfidf_matrix)
    
    return cosine_similarities
    
def generate_similarity_rank(cosine_similarities, labels, min_strength):
    data = pd.DataFrame()
    # Print the cosine similarity matrix
    for i in tqdm(range(len(cosine_similarities))):
        for j in range(i + 1, len(cosine_similarities)):
            # print(f"Similarity between list {i} and list {j}: {cosine_similarities[i][j]}")
            tmp = pd.DataFrame()
            if cosine_similarities[i][j] > min_strength and labels[i] != labels[j]:
                tmp['content_1'] = [labels[i]]
                tmp['content_2'] = [labels[j]]
                tmp['similarity'] = [cosine_similarities[i][j]]
                data = data.append(tmp)
                
    data = data.sort_values(by=['similarity'], ascending = False)

    return data

def find_similar_content(cosine_similarities, labels, content, min_strength):
    data = pd.DataFrame()
    
    for i in range(len(cosine_similarities)):

        # Temporary dataframe 
        tmp = pd.DataFrame()

        for j in range(i + 1, len(cosine_similarities)):

            if cosine_similarities[i][j] > min_strength and labels[i] == content:
                tmp['content_1'] = [labels[i]]
                tmp['content_2'] = [labels[j]]
                tmp['similarity'] = [cosine_similarities[i][j]]

                data = data.append(tmp)

    data = data.sort_values(by = ['similarity'], ascending = False)
    
    return data

def cluster_data(document_list, number_of_clusters):
    # Fit and transform the vectorizer on the document list
    vectorized_docs = vectorizer.fit_transform(document_list)

    # Apply dimensionality reduction to reduce the vectorized documents to two or three dimensions
    pca = PCA(n_components=2)  # or n_components=3 for 3D visualization
    reduced_docs = pca.fit_transform(vectorized_docs.toarray())

    # Perform K-means clustering
    kmeans = KMeans(n_clusters=number_of_clusters)
    cluster_labels = kmeans.fit_predict(reduced_docs)
    
    return [reduced_docs, cluster_labels]

def generate_clustered_table(reduced_docs, labels, cluster_labels):
    data = pd.DataFrame()

    for n, loc in enumerate(reduced_docs):
        tmp = pd.DataFrame()
        
        loc = str(loc).replace('[','').replace(']','').strip().replace('  ', ' ').replace('  ', ' ')
        tmp['x'] = [loc.split(' ')[0]]
        tmp['y'] = [loc.split(' ')[1]]
        tmp['content'] = [labels[n]]
        tmp['topic'] = [cluster_labels[n]]
        
        data = data.append(tmp)
    
    data = data.sort_values(by=['topic'])
    return data

def draw_viz(reduced_docs, cluster_labels):
    # Visualize the clustered data
    plt.style.use('dark_background')

    fig, ax = plt.subplots()
    scatter = ax.scatter(reduced_docs[:, 0], reduced_docs[:, 1], c=cluster_labels, cmap='BuPu')

    plt.title('Clustered Documents')
    plt.xlabel('Dimension 1')
    plt.ylabel('Dimension 2')
    plt.colorbar(scatter)

    plt.show()

def draw_viz_with_topics(reduced_docs, cluster_labels, keywords):
    plt.style.use('dark_background')

    fig, ax = plt.subplots(figsize=(70, 70))  # Adjust the figsize as per your desired resolution

    scatter = ax.scatter(reduced_docs[:, 0], reduced_docs[:, 1], c=cluster_labels, cmap='hsv')

    for i, txt in enumerate(keywords):
        ax.annotate(txt[0], (reduced_docs[i, 0], reduced_docs[i, 1]), fontsize=8)

    plt.title('Clustered Documents with First Keyword')
    plt.xlabel('Dimension 1')
    plt.ylabel('Dimension 2')
    plt.colorbar(scatter)

    plt.show()

def elbow_method():
    max_clusters = 10
    wcss = elbow_method(reduced_docs, max_clusters)
    plt.plot(range(1, max_clusters + 1), wcss, marker='o')
    plt.title('Elbow Method')
    plt.xlabel('Number of Clusters')
    plt.ylabel('WCSS')
    plt.show()

def clean_keywords(keywords):
    local_stopwords = ['the', 'rok', 'Rok', 'The']
    cleaned_keywords = []
    for keyword_list in keywords:
        cleaned_list = []
        for keyword in keyword_list:
            cleaned_keyword = re.sub(r'[^a-zA-Z0-9ąćęłńóśźżĄĆĘŁŃÓŚŹŻ\s]', '', keyword)
            if cleaned_keyword not in local_stopwords:
                cleaned_list.append(cleaned_keyword)
        cleaned_keywords.append(cleaned_list)
    return cleaned_keywords




# Methodology tests

Preparing data and variables

In [5]:
output = topic_modeling(data, 'content', 'title')
topic_list, labels = output[0], output[1]

100%|██████████| 2092/2092 [26:32<00:00,  1.31it/s]


In [6]:
keywords = generate_keywords(topic_list)
#print(keywords) 
# Convert each list of words into a string
document_list = [' '.join(words) for words in keywords]
sim_matrix = generate_similarity_matrix(document_list)

output = cluster_data(document_list, 8)
reduced_docs, cluster_labels = output[0], output[1]

Creating similarity rank

In [7]:
sim_rank = generate_similarity_rank(sim_matrix, labels, 0.2)
sim_rank

100%|██████████| 800/800 [00:52<00:00, 15.24it/s] 


,content_1,content_2,similarity
0,Rusza głosowanie na 4. Rapowy Top Wszech Czasó...,"Spieszcie się, to już ostatnie godziny głosowa...",0.690673
0,TOP 25 najlepszych polskich płyt 2021 roku,TOP 25 najlepszych zagranicznych utworów 2021 ...,0.684535
0,"Tymek & Urbański – trzeba mieć odwagę, żeby si...",Tymek & Urbański: „Nasz album jest ponadczasow...,0.581557
0,Nicolas Cage stał się naczelnym trollem kina. ...,"Nicolas Cage gra Nicolasa Cage'a. I fajnie, al...",0.579267
0,50 najlepszych filmów w historii polskiego kin...,50 najlepszych filmów w historii polskiego kin...,0.532914
...,...,...,...
0,Z małej mazowieckiej wsi na bestsellerową galę...,"Tu, żeby dostać nagrodę, trzeba wiedzieć, jak ...",0.200794
0,Prime Video wypuszcza fabularny serial o... br...,HBO Max w Polsce już 8 marca? To bardzo prawdo...,0.200555
0,"Macie chwilę, żeby porozmawiać o inwestowaniu ...",Tydzień w technologicznym świecie: prawdziwe d...,0.200192
0,Cały ten Merge. O co chodzi w największej tech...,Tydzień w technologicznym świecie: możliwy zmi...,0.200143


Find similar content

In [ ]:
title = 'Nie okazuj emocji i nie proś o pomoc. O stereotypach dotyczących męskiego zdrowia psychicznego (ROZMOWA)'
similars = find_similar_content(sim_matrix, labels, title, 0.1)
similars

Clustered table

In [ ]:
clustered_table = generate_clustered_table(reduced_docs, labels, cluster_labels)
clustered_table

Clustered viz

In [8]:
#draw_viz(reduced_docs, cluster_labels)
keywords = clean_keywords(keywords)
print(keywords)

[['dom', 'miasteczko', 'efekt', 'ciekawy', 'praca', 'amerykański', 'czysty', 'spokojnie', 'mężczyzna', 'pojęcie', 'martwić', 'kochanie', 'warner', 'wizja', 'frank', 'victory', 'zresztą', 'idealny', '50', 'film', 'alica', 'pugh', 'dobry', 'scena', 'drążyć', 'wiadomo', 'martwić', 'ambitny', 'wojna'], ['sierpień', 'hip', 'hop', 'festival', 'kuba', 'quebonafide', 'grabowski', '13', 'polish', 'zatem', 'wrzesień', 'Polska', 'zielonka', 'trasa', '4', '2', 'metr', 'sierpień', 'cztery', 'województwo', 'quebo', 'występ', 'koncert', 'gig', 'weekend', 'grabowski', 'wielki', 'biznees', 'czekać', 'dotyczyć'], ['trasa', 'relations', 'psycho', 'duet', 'muzyczny', 'skoro', 'temat', 'filip', 'pewien', 'dzień', 'tour', 'kolejny', 'will', 'zostać', 'pierwszy', 'trasa', 'mówić', 'soma', 'say', 'wspólny', 'taco', 'quebo', 'młody', 'urokliwy', 'taconafide', 'płyta', 'soma', 'miesiąc', 'streaming'], ['wilka', 'wdz', '1988', 'wyglądać', 'producencki', 'trzydzieści', 'scena', 'wyróżniać', 'dobry', 'dom', 'wilka

In [ ]:
draw_viz_with_topics(reduced_docs, cluster_labels, keywords)

#Keywords exploratory analysis


In [ ]:
#Top keywords for each cluster:
import itertools
from collections import Counter

def aggregate_top_keywords(keywords, cluster_labels, top_n=10):
    # Create a dictionary to store the aggregated keywords for each cluster
    cluster_keywords = {}

    # Iterate over the cluster labels
    for cluster_label in set(cluster_labels):
        # Get the keywords associated with the current cluster label
        cluster_keywords_list = [kw for kw, lbl in zip(keywords, cluster_labels) if lbl == cluster_label]

        # Flatten the list of keywords
        flattened_keywords = list(itertools.chain.from_iterable(cluster_keywords_list))

        # Count the frequency of each keyword
        keyword_counts = Counter(flattened_keywords)

        # Get the top N keywords for the current cluster
        top_keywords = keyword_counts.most_common(top_n)

        # Store the top keywords in the cluster_keywords dictionary
        cluster_keywords[cluster_label] = top_keywords

    return cluster_keywords

top_keywords_per_cluster = aggregate_top_keywords(keywords, cluster_labels, top_n=10)

# Print the top keywords for each cluster
for cluster_label, top_keywords in top_keywords_per_cluster.items():
    print(f"Cluster {cluster_label}:")
    for keyword, count in top_keywords:
        print(f"- {keyword} ({count} occurrences)")
    print()


In [ ]:
#Check for unique keywords for each cluster

from collections import Counter

def unique_cluster_keywords(cleaned_keywords, cluster_labels):
    # Create a dictionary to store the keywords for each cluster
    cluster_keywords = {}

    # Iterate over the cluster labels
    for cluster_label in set(cluster_labels):
        # Get the cleaned keywords associated with the current cluster label
        cluster_keywords_list = [kw for kw, lbl in zip(cleaned_keywords, cluster_labels) if lbl == cluster_label]

        # Flatten the list of cleaned keywords
        flattened_keywords = [kw for sublist in cluster_keywords_list for kw in sublist]

        # Count the frequency of each cleaned keyword across all clusters
        keyword_counts = Counter(flattened_keywords)

        # Get the unique keywords that do not appear in other clusters
        unique_keywords = [kw for kw, count in keyword_counts.items() if count == 1 and all(kw not in other_kw for other_kw in flattened_keywords)]

        # Store the unique keywords in the cluster_keywords dictionary
        cluster_keywords[cluster_label] = unique_keywords

    return cluster_keywords


unique_keywords_per_cluster = unique_cluster_keywords(keywords, cluster_labels)

# Print the unique keywords for each cluster
for cluster_label, unique_keywords in unique_keywords_per_cluster.items():
    print(f"Cluster {cluster_label}:")
    print(", ".join(unique_keywords))
    print()


In [ ]:
#All keywords for each cluster

from collections import defaultdict

def get_cluster_keywords(cleaned_keywords, cluster_labels):
    cluster_keywords = defaultdict(set)
    
    for keyword, label in zip(cleaned_keywords, cluster_labels):
        cluster_keywords[label].update(keyword)
        
    return cluster_keywords


def print_cluster_keywords(cluster_keywords):
    for cluster_label, keywords in cluster_keywords.items():
        print(f"Cluster {cluster_label}:")
        for keyword in keywords:
            print(keyword)
        print()

cluster_keywords = get_cluster_keywords(keywords, cluster_labels)
print_cluster_keywords(cluster_keywords)


In [ ]:
#loop through clusters
for i in range(5, 15):
  output = cluster_data(document_list, i)
  reduced_docs, cluster_labels = output[0], output[1]
  draw_viz_with_topics(reduced_docs, cluster_labels, keywords)
  #calculate the mean square root sum (elbow)